# Raibert-Style Hopper Simulation

This notebook demonstrates a one-legged hopping robot controlled using the Raibert control algorithm.

The hopper uses three main control strategies:
1. **Foot placement** for forward speed control
2. **Thrust timing** for hopping height
3. **Body attitude** for stability during stance

In [ ]:
# Install dependencies (uncomment if running on Colab)
# !pip install numpy matplotlib scipy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, cumulative_trapezoid
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from dataclasses import dataclass
import time
import copy

## Parameters Class

In [ ]:
@dataclass
class Parameters:
    """Parameters class for hopper simulation"""
    # Physical parameters
    m: float = 10.0        # mass of the body
    m_l: float = 1.0       # mass of the leg
    J: float = 10.0        # moment of inertia of the body
    J_l: float = 1.0       # moment of inertia of the leg
    g: float = 9.8         # gravity
    k_l: float = 1e3       # spring constant of leg spring
    k_stop: float = 2e5    # spring constant of leg stop
    b_stop: float = 1e3    # damping constant of leg stop
    k_g: float = 1e4       # spring constant of the ground
    b_g: float = 300.0     # damping constant of the ground
    r_s0: float = 1.0      # rest length of the leg spring
    l_1: float = 0.5       # distance from the foot to the com of the leg
    l_2: float = 0.4       # distance from the hip to the com of the body

    # FSM states
    FSM_COMPRESSION: int = 0
    FSM_THRUST: int = 1
    FSM_LOADING: int = 99
    FSM_FLIGHT: int = 2
    FSM_NUM_STATES: int = 3

    # State parameters
    fsm_state: int = 2  # FSM_FLIGHT - usually we start in the air
    t_state_switch: float = 0.0
    x_dot_des: float = 0.0
    T_s: float = 0.425  # Stance time estimate
    T_compression: float = 0.0
    t_thrust_on: float = 0.0
    T_MAX_THRUST_DUR: float = 0.425 * 0.35

    toString: str = '1: x foot 2: y foot 3: abs angle leg (vert) 4: abs angle body (vert) 5:leg length'


def hopperParameters():
    """function p = hopperParameters()"""
    return Parameters()

## Control System

In [ ]:
def hopperStateControl(t, q, param):
    """function [u,internalStruct] = hopperStateControl(t,q,param)"""
    THRUST = param.FSM_THRUST
    COMPRESSION = param.FSM_COMPRESSION
    FLIGHT = param.FSM_FLIGHT

    # Control gains
    k_fp = 150.0   # foot placement gain
    b_fp = 15.0    # foot placement damping
    k_att = 150.0  # attitude gain
    b_att = 15.0   # attitude damping
    k_xdot = 0.02  # forward speed gain
    thrust = 0.035 * param.k_l  # thrust force
    thr_z_low = 0.01
    u_retract = -0.1 * param.k_l

    if param.T_s == 0:
        print('Warning: using default T_s.')
        T_s = 0.425
    else:
        T_s = param.T_s

    u = np.array([0.0, 0.0])
    a_des = 0
    
    d_xfoot_dt = q[5]
    y_foot = q[1]
    a = q[2]
    dadt = q[7]
    b = q[3]
    dbdt = q[8]
    l = q[4]
    dldt = q[9]
    stance_ang_des = a / 2

    if param.fsm_state == THRUST:
        u[0] = thrust
        u[1] = -k_att * (b - stance_ang_des) - b_att * dbdt
    elif param.fsm_state == COMPRESSION:
        u[1] = -k_att * (b - stance_ang_des) - b_att * dbdt
    elif param.fsm_state == FLIGHT:
        d_xbody_dt = d_xfoot_dt + dldt * np.sin(a) + l * np.cos(a) * dadt + param.l_2 * np.cos(b) * dbdt
        a_des = -np.arcsin((1 * d_xbody_dt * T_s / 2 + k_xdot * (d_xbody_dt - param.x_dot_des)) / l)
        if np.isnan(a_des):
            print('warning! nan in a_des. an error has occurred.')
        if y_foot > thr_z_low:
            u[1] = k_fp * (a - a_des) + b_fp * (dadt)

    # Thrust duration limiting
    if param.fsm_state == THRUST:
        if t - param.t_thrust_on > param.T_MAX_THRUST_DUR:
            u[0] = 0
    
    return u, a_des

## Dynamics

In [ ]:
def hopperDynamicsFwd(t, q, p_obj):
    """function structOut = hopperDynamicsFwd(t,q,p_obj)"""
    u, a_des = hopperStateControl(t, q, p_obj)
    R = q[4] - p_obj.l_1
    s1 = np.sin(q[2])
    c1 = np.cos(q[2])
    s2 = np.sin(q[3])
    c2 = np.cos(q[3])

    r_sd = p_obj.r_s0 - q[4]
    if r_sd > 0:
        F_k = p_obj.k_l * r_sd + u[0]
    else:
        F_k = p_obj.k_stop * r_sd + u[0] - p_obj.b_stop * q[9]

    if q[1] < 0:
        F_x = -p_obj.b_g * q[5]
        F_z = p_obj.k_g * (-q[1])
        F_z = F_z + max(-p_obj.b_g * q[6], 0.0)
    else:
        F_x = 0.0
        F_z = 0.0

    a = p_obj.l_1 * F_z * s1 - p_obj.l_1 * F_x * c1 - u[1]

    M = np.array([
        [-p_obj.m_l * R, 0, (p_obj.J_l - p_obj.m_l * R * p_obj.l_1) * c1, 0, 0],
        [0, p_obj.m_l * R, (p_obj.J_l - p_obj.m_l * R * p_obj.l_1) * s1, 0, 0],
        [p_obj.m * R, 0, (p_obj.J_l + p_obj.m * R * q[4]) * c1, p_obj.m * R * p_obj.l_2 * c2, p_obj.m * R * s1],
        [0, -p_obj.m * R, (p_obj.J_l + p_obj.m * R * q[4]) * s1, p_obj.m * R * p_obj.l_2 * s2, -p_obj.m * R * c1],
        [0, 0, p_obj.J_l * p_obj.l_2 * np.cos(q[2] - q[3]), -p_obj.J * R, 0]
    ])

    eta = np.array([
        a * c1 - R * (F_x - F_k * s1 - p_obj.m_l * p_obj.l_1 * q[7] * q[7] * s1),
        a * s1 + R * (p_obj.m_l * p_obj.l_1 * q[7] * q[7] * c1 + F_z - F_k * c1 - p_obj.m_l * p_obj.g),
        a * c1 + R * F_k * s1 + p_obj.m * R * (q[4] * q[7] * q[7] * s1 + p_obj.l_2 * q[8] * q[8] * s2 - 2 * q[9] * q[7] * c1),
        a * s1 - R * (F_k * c1 - p_obj.m * p_obj.g) - p_obj.m * R * (2 * q[9] * q[7] * s1 + q[4] * q[7] * q[7] * c1 + p_obj.l_2 * q[8] * q[8] * c2),
        a * p_obj.l_2 * np.cos(q[2] - q[3]) - R * (p_obj.l_2 * F_k * np.sin(q[3] - q[2]) + u[1])
    ])

    qdd = np.linalg.solve(M, eta)
    xdot = np.concatenate([q[5:10], qdd])

    structOut = {}
    structOut['stated'] = xdot
    structOut['u'] = u
    structOut['a_des'] = a_des
    structOut['r_sd'] = r_sd
    structOut['fsm_state'] = p_obj.fsm_state

    return structOut


def hopperDynamics(t, q, p_obj):
    """function qdot = hopperDynamics(t,q,p_obj)"""
    structFwd = hopperDynamicsFwd(t, q, p_obj)
    qdot = structFwd['stated']
    return qdot

## Event Detection (State Machine Transitions)

In [ ]:
def eventsHopperControl(t, q, param):
    """function [value,isterminal,direction] = eventsHopperControl(t,q,param)"""
    COMPRESSION = param.FSM_COMPRESSION
    THRUST = param.FSM_THRUST
    FLIGHT = param.FSM_FLIGHT

    y_foot = q[1]
    ddt_leg = q[9]
    l = q[4]
    thresh_leg_extended = 0.0001

    if param.fsm_state == COMPRESSION:
        value = ddt_leg  # go from compression to thrust when leg stops compressing
    elif param.fsm_state == THRUST:
        value = -(param.r_s0 - l) - thresh_leg_extended  # when leg is fully extended
    elif param.fsm_state == FLIGHT:
        value = -y_foot  # touchdown
    else:
        value = 1.0  # Should not reach here

    return value

## Visualization

In [ ]:
def draw(p_obj, t, q):
    """function draw(p_obj,t,q)"""
    l_1 = p_obj.l_1
    l_2 = p_obj.l_2

    if not hasattr(draw, 'hFig'):
        draw.hFig = None
        draw.xcamera = 0

    if draw.hFig is None:
        draw.hFig = plt.figure(25, figsize=(10, 6))
        draw.brick1 = np.array([[0.9, 1.05, 1.05, 0.9, 0.9], [0.15, 0.15, 0.0, 0.0, 0.15]])
        draw.brick2 = np.array([[-1, 0], [0, 1]]) @ draw.brick1
        draw.beam = np.array([[-1, -1, 1, 1], [0.15, 0.175, 0.175, 0.15]])
        draw.comp = np.array([[0.25, 0.25, 0.025, 0.025, -0.025, -0.025, -0.25, -0.25],
                               [0.15, 0.3, 0.3, 0.15, 0.15, 0.3, 0.3, 0.15]])
        draw.body = np.hstack([draw.brick1, draw.brick2, draw.beam, draw.comp])
        draw.body = draw.body - np.array([[0], [0.16]]) @ np.ones((1, draw.body.shape[1]))
        draw.leg = np.array([[-0.015, -0.04, -0.04, 0.04, 0.04, 0.015, -0.015],
                              [-0.25, -0.25, 0.525, 0.525, -0.25, -0.25, -0.25]])
        draw.leg = draw.leg - np.array([[0], [0.16]]) @ np.ones((1, draw.leg.shape[1]))
        draw.foot = np.array([[0.0, -0.03, -0.03, 0.015, 0.015, -0.025, -0.025, 0.025, 0.025, -0.015, -0.015, 0.03, 0.03, 0.0],
                               [0.0, 0.05, 0.14, 0.14, 1.675, 1.675, 1.725, 1.725, 1.675, 1.675, 0.14, 0.14, 0.05, 0.0]])

    hip = np.array([0, q[4]])
    rot = np.array([[np.cos(q[2]), np.sin(q[2])], [-np.sin(q[2]), np.cos(q[2])]])
    hip = rot @ hip + np.array([q[0], q[1]])
    cop = np.array([q[0], q[1]])

    Foot = rot @ draw.foot + np.array([[q[0]], [q[1]]])
    Leg = rot @ draw.leg + hip.reshape(2, 1)
    rot = np.array([[np.cos(q[3]), np.sin(q[3])], [-np.sin(q[3]), np.cos(q[3])]])
    Body = rot @ draw.body + hip.reshape(2, 1)

    plt.figure(draw.hFig.number)
    plt.clf()

    draw.xcamera = np.mean([hip[0], draw.xcamera])
    xmin = draw.xcamera - 5
    xmax = draw.xcamera + 5
    ymin = -0.25
    ymax = 3.0

    plt.plot([xmin, xmax], [0, 0], 'k-')
    plt.axis('equal')
    plt.axis([xmin, xmax, ymin, ymax])

    plt.plot(Foot[0, :], Foot[1, :], 'k-')
    plt.plot(Leg[0, :], Leg[1, :], 'k-')
    plt.plot(Body[0, :], Body[1, :], 'k-')
    c = 0.75
    plt.fill(Foot[0, :], Foot[1, :], color=[c, c, c])
    plt.fill(Leg[0, :], Leg[1, :], color=[c, c, c])
    plt.fill(Body[0, :], Body[1, :], color=[c, c, c])

    plt.title(f't={t:.3f} s')

## Main Simulation Function

In [ ]:
def call_hopper(tstart=0, tfinal=5, x_dot_des=2.0, y0=None, sr_sim=1000, animate=True):
    """Main simulation script
    
    Args:
        tstart (float): Start time for simulation (default: 0)
        tfinal (float): Final time for simulation (default: 5)
        x_dot_des (float): Desired forward velocity in m/s (default: 2.0)
        y0 (np.array): Initial state vector (default: standard starting state)
        sr_sim (int): Simulation sample rate (default: 1000)
        animate (bool): Whether to create animation (default: True)
    
    Returns:
        tuple: (tout, yout, State, p, anim) where anim is the FuncAnimation object
    """
    # Set default initial conditions if not provided
    if y0 is None:
        y0 = np.array([0.0, 0.4, 0.01, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0])

    p = hopperParameters()
    p.x_dot_des = x_dot_des
    p.t_state_switch = tstart
    tout = []
    yout = []
    teout = []
    yeout = []
    ieout = []
    cstate = []
    at_des = []
    extra_states = []
    tic = time.time()

    print(f"Running simulation with desired velocity: {x_dot_des} m/s")
    
    while tfinal - tstart > 1e-3:
        p_snapshot = copy.deepcopy(p)

        def event_func(t, y):
            return eventsHopperControl(t, y, p_snapshot)
        
        event_func.terminal = True
        event_func.direction = 1

        sol = solve_ivp(
            lambda t, y: hopperDynamics(t, y, p_snapshot),
            [tstart, tfinal],
            y0,
            method='Radau',
            events=event_func,
            rtol=1e-8,
            atol=1e-8,
            max_step=0.05,
            dense_output=True,
            vectorized=False
        )

        t = np.concatenate([np.arange(tstart, sol.t[-1], 1/sr_sim), [sol.t[-1]]])
        states = sol.sol(t)
        states = states.T
        t = t[:-1]
        tout.append(t)

        if len(sol.t_events[0]) > 0:
            cstate.append(np.repeat(p.fsm_state, len(t)))
        else:
            if len(cstate) > 0:
                cstate.append(np.repeat(cstate[-1][-1], len(t)))
            else:
                cstate.append(np.repeat(p.fsm_state, len(t)))

        yout.append(states[:-1, :])

        if len(sol.t_events[0]) > 0:
            teout.append(sol.t_events[0])
        if len(sol.y_events[0]) > 0:
            yeout.append(sol.y_events[0])
        if len(sol.t_events[0]) > 0:
            ieout.append(np.ones(len(sol.t_events[0])))

        if len(sol.y_events[0]) > 0 and p.fsm_state == p.FSM_FLIGHT:
            strOut = hopperDynamicsFwd(sol.t_events[0][0], sol.y_events[0][0], p)
            at_des.append([sol.t_events[0][0], strOut['a_des']])

        int_e = []
        for iint in range(len(t)):
            int_states = hopperDynamicsFwd(t[iint], states[iint, :], p)
            int_e.append([int_states['u'][0], int_states['u'][1], int_states['fsm_state']])
        int_e = np.array(int_e)
        extra_states.append(int_e)

        if p.fsm_state == p.FSM_THRUST:
            p.T_s = (t[-1] - tstart) + p.T_compression
        elif p.fsm_state == p.FSM_COMPRESSION:
            p.T_compression = t[-1] - tstart

        if p.fsm_state == p.FSM_COMPRESSION:
            p.t_thrust_on = t[-1]

        if len(sol.t_events[0]) > 0:
            p.fsm_state = p.fsm_state + 1
            p.fsm_state = p.fsm_state % p.FSM_NUM_STATES

        y0 = yout[-1][-1, :]
        tstart = t[-1]

    toc = time.time()
    print(f'Simulation elapsed time: {toc - tic:.2f} seconds')

    # Concatenate all accumulated arrays
    tout = np.concatenate(tout)
    yout = np.vstack(yout)
    cstate = np.concatenate(cstate)
    extra_states = np.vstack(extra_states)
    if len(at_des) > 0:
        at_des = np.array(at_des)

    # Plot results
    fig1 = plt.figure(figsize=(10, 6))
    d_xfoot_dt = yout[:, 5]
    dldt = yout[:, 9]
    l = yout[:, 4]
    dadt = yout[:, 7]
    a = yout[:, 2]
    b = yout[:, 3]
    dbdt = yout[:, 8]
    d_xbody_dt = d_xfoot_dt + dldt * np.sin(a) + l * dadt * np.cos(a) + p.l_2 * dbdt * np.cos(b)
    
    plt.subplot(2, 1, 1)
    plt.plot(tout, d_xbody_dt, label='Actual velocity')
    xl = plt.xlim()
    plt.plot([xl[0], xl[1]], [p.x_dot_des, p.x_dot_des], 'r--', label='Desired velocity')
    plt.ylabel('Body velocity (m/s)')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(2, 1, 2)
    plt.plot(tout, yout[:, 2], label='Actual leg angle')
    if len(at_des) > 0:
        plt.plot(at_des[:, 0], at_des[:, 1], 'rx', label='Desired leg angle at touchdown')
    plt.ylabel('Leg angle (rad)')
    plt.xlabel('Time (s)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Create state dictionary
    State = {}
    State['x_foot'] = yout[:, 0]
    State['z_foot'] = yout[:, 1]
    State['phi_leg'] = yout[:, 2]
    State['phi_body'] = yout[:, 3]
    State['len_leg'] = yout[:, 4]
    State['ddt_x_foot'] = yout[:, 5]
    State['ddt_z_foot'] = yout[:, 6]
    State['ddt_phi_leg'] = yout[:, 7]
    State['ddt_phi_body'] = yout[:, 8]
    State['ddt_len_leg'] = yout[:, 9]
    State['u'] = extra_states[:, 0:2]
    State['fsm_state'] = extra_states[:, 2]

    anim = None
    if animate:
        # Animation setup
        sr_video = 24
        ds = round(sr_sim / sr_video)
        tout_vid = tout[::ds]
        yout_vid = yout[::ds, 0:5]

        print(f"Creating animation with {len(yout_vid)} frames...")

        # Create animation figure
        anim_fig = plt.figure(25, figsize=(10, 6))

        def animate(i):
            """Animation function for each frame"""
            draw(p, tout_vid[i], yout_vid[i, 0:5])
            return []

        # Create animation
        anim = FuncAnimation(anim_fig, animate, frames=len(yout_vid),
                            interval=1000/sr_video, blit=False, repeat=True)
        
        print("Animation created successfully!")

    return tout, yout, State, p, anim

## Run the Simulation

In [ ]:
# Run simulation with desired forward velocity of 2 m/s
tout, yout, State, p, anim = call_hopper(tfinal=5, x_dot_des=2.0)

print(f"\nSimulation complete!")
print(f"Final time: {tout[-1]:.4f} s")
print(f"Final position: x={yout[-1, 0]:.2f} m")
print(f"Distance traveled: {yout[-1, 0] - yout[0, 0]:.2f} m")

## Display Animation

Run this cell to display the animation as an HTML5 video in the notebook.

In [ ]:
# Display animation as HTML5 video
if anim is not None:
    HTML(anim.to_html5_video())
else:
    print("No animation available. Run the simulation first.")

## Experiment with Different Parameters

Try different desired velocities to see how the hopper adapts its gait!

In [ ]:
# Try a faster velocity
tout_fast, yout_fast, State_fast, p_fast, anim_fast = call_hopper(tfinal=5, x_dot_des=3.5)

# Display the animation
if anim_fast is not None:
    HTML(anim_fast.to_html5_video())

## About the Raibert Controller

This simulation implements the classic Raibert hopping controller with three key control strategies:

1. **Forward Speed Control via Foot Placement**: The foot is placed ahead or behind the body to control forward velocity. The desired foot placement angle is computed based on the current velocity error and the estimated stance time.

2. **Hopping Height Control via Thrust Timing**: A thrust force is applied during the stance phase to maintain hopping height. The thrust duration is limited to prevent over-extension.

3. **Body Attitude Control**: During stance, the body angle is controlled to maintain stability and ensure proper energy transfer.

The controller uses a finite state machine (FSM) with three states:
- **COMPRESSION**: Leg is compressing after touchdown
- **THRUST**: Active thrust is applied to the leg
- **FLIGHT**: Robot is in the air

For more information, see:
- Raibert, M. H. (1986). Legged Robots That Balance. MIT Press.
- Tedrake, R. (2024). Underactuated Robotics. MIT Press.